# Deploy ERAS Package from AWS Marketplace

**ERAS - Easy Reading Analyzer for Spanish** is an easy reading analyzer for Spanish texts. With four global indexes and thirty subindexes, it evaluates text linguistic features and allows to easily detect readability improvement areas. 

This sample notebook shows you how to deploy ERAS using Amazon SageMaker.

## Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to ERAS. If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)

## Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Visualize output](#D.-Visualize-output)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Perform batch inference](#3.-Perform-batch-inference)
4. [Clean-up](#4.-Clean-up)
   1. [Delete the model](#A.-Delete-the-model)
   2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Subscribe to the model package

To subscribe to the model package:

1. Open the model package listing page ERAS.
2. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
3. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms.
4. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [ ]:
import json

import boto3

import sagemaker as sage
from sagemaker import ModelPackage, get_execution_role
from sagemaker.huggingface.model import HuggingFacePredictor

In [ ]:
model_package_arn = "arn:aws:sagemaker:eu-west-1:...:model-package/eras"

In [ ]:
role = get_execution_role()

sagemaker_session = sage.Session()

bucket = sagemaker_session.default_bucket()
runtime = boto3.client("runtime.sagemaker")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/whatis.html)
.

In [ ]:
model_name = "easy_reading_analyzer"

content_type = "application/json"

real_time_inference_instance_type = "ml.m5.xlarge"

### A. Create an endpoint

In [ ]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

Once endpoint has been created, you would be able to perform real-time inference.

### B. Create input payload

In [ ]:
with open("data/input/data_input.json") as fp:
    payload = json.load(fp)

### C. Perform real-time inference

In [ ]:
predictor = HuggingFacePredictor(model_name)
response = predictor.predict(payload)

### D. Visualize output

In [ ]:
r = json.load(response["Body"])

if "html" in json.loads(r["body"]):
    html_content = json.loads(r["body"])["html"]

    with open(
        "path_to/output.html",
        "w",
        encoding="utf-8",
    ) as f:
        f.write(html_content)
        print("HTML saved")
    print("Metrics: ", json.loads(r["body"])["summary"])

else:
    print("No 'html' field found in response body")
    print("Metrics: ", json.loads(r["body"]))

### E. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 3. Perform batch inference

In this section, you will perform batch inference using multiple input payloads together. 

In [ ]:
transform_input_data = "data/input/input.json"

with open(transform_input_data) as fp:
    payload = json.load(fp)

In [ ]:
# upload the batch-transform job input files to S3
transform_input = sagemaker_session.upload_data(
    transform_input_data, key_prefix=model_name
)
print("Transform input uploaded to " + transform_input)

In [ ]:
# Run the batch-transform job
transformer = model.transformer(1, "ml.m5.large")
transformer.transform(transform_input, content_type="application/json")
transformer.wait()

In [ ]:
# output is available on following path
transformer.output_path  # noqa B018

In [ ]:
!aws s3 cp $transformer.output_path . --recursive

output_path = f"{transform_input_data.rsplit('/')[-1]}.out"
with open(output_path, "r") as fp:
    response = json.load(fp)

In [ ]:
res = json.load(response["body"])


if "html" in json.load(response["body"]):
    html_content_batch = json.loads(r["body"])["html"]

    with open(
        "data/output.html",
        "w",
        encoding="utf-8",
    ) as f:
        f.write(html_content_batch)
        print("HTML saved")
    print("Metrics: ", json.loads(r["body"])["summary"])

else:
    print("No 'html' field found in response body")
    print("Metrics: ", json.loads(r["body"]))

## 4. Clean-up

### A. Delete the model

In [ ]:
model.delete_model()

### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any deployable model created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model.

**Steps to unsubscribe to product from AWS Marketplace:**

1. Navigate to **Machine Learning** tab on Your Software subscriptions page
2. Locate the listing that you want to cancel the subscription for, and then choose *Cancel Subscription* to cancel the subscription.